<a href="https://colab.research.google.com/github/LuisHVieira/tcc/blob/main/modelBuilder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install livelossplot

In [ ]:
import tensorflow as tf
from keras import backend
from keras.optimizers import Adam, Optimizer
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.imagenet_utils import preprocess_input
from keras import Model
from keras.layers import Dense
from keras.layers import GlobalAveragePooling2D
from keras.layers import GlobalAvgPool1D
from livelossplot import PlotLossesKeras
from enum import Enum
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, recall_score, precision_score, accuracy_score, hamming_loss
import matplotlib.pyplot as plt
import seaborn as sns
import time, datetime

In [ ]:
class Architecture(Enum):
  VGG16 = 'vgg16'
  RESNET50V2 = 'resnet50'
  MOBILENETV2 = 'mobilenet'

In [ ]:
class Datas():

  list_generate = []
  list_data = []


  def __init__(self, target_size = (224, 224),  batch_size = 32) -> None:
    self.target_size = target_size
    self.batch_size = batch_size




  def data_augumentation(self) -> None:

    generate_train = ImageDataGenerator(rotation_range= 7, horizontal_flip=True, shear_range=0.2, height_shift_range=0.07, zoom_range= 0.2, preprocessing_function=preprocess_input)
    generate_test = ImageDataGenerator(preprocessing_function=preprocess_input)

    self.list_generate.clear()
    self.list_generate.append(generate_train)
    self.list_generate.append(generate_train)




  def load_dataSet(self, paths_datas:list) -> None:

    generate_train, generate_test = self.list_generate
    path_train, path_val, path_test = paths_datas

    try:
      data_train = generate_train.flow_from_directory( path_train, target_size=self.target_size, batch_size=self.batch_size, shuffle=True)
      data_val =  generate_train.flow_from_directory( path_val, target_size=self.target_size,  batch_size=self.batch_size, shuffle=True)
      data_test =  generate_test.flow_from_directory( path_test, target_size=self.target_size, class_mode=None, batch_size=1, shuffle=False)

      self.list_data.clear()
      self.list_data.append(data_train)
      self.list_data.append(data_val)
      self.list_data.append(data_test)

    except NotADirectoryError as not_dir:
      print(not_dir)


In [ ]:
class Builder():

  input_shape = (224, 224, 3)
  callbacks = []
  n_steps = 0
  n_val_steps = 0
  total_time = 0
  model = None
  history = None

  def __init__(self, data:Datas, architecture:Architecture, n_epochs = 100,  optimizer = Adam(learning_rate=0.0001), fine_tune=0) -> None:
    self.batch_size = data.batch_size
    self.data_train = data.list_data[0]
    self.data_val = data.list_data[1]
    self.n_classes = self.data_train.num_classes
    self.architecture = Architecture(architecture)
    self.n_epochs = n_epochs
    self.optimizer = optimizer
    self.fine_tune = fine_tune
    self.set_callbacks()
    self.set_steps()


  def set_callbacks(self) -> None:

    base_name = 'bestWeights' + str(self.architecture.name)
    ext = '.weights.best.hdf5'

    name_file = base_name + ext if self.fine_tune == 0 else base_name + '_with_fine_tune' + ext

    self.callbacks =  [
                        PlotLossesKeras(),
                        EarlyStopping(monitor = 'val_loss', patience = 5 ),
                        ModelCheckpoint( filepath = name_file, save_best_only=True, save_weights_only=True, verbose=1)
                      ]

  def set_steps(self) -> None:

    self.n_steps = self.data_train.samples // self.batch_size
    self.n_val_steps = self.data_val.samples // self.batch_size


  def summary_model(self) -> None:

    self.model._name = str(self.architecture.name) if self.fine_tune == 0 else str(self.architecture.name) + '_With_Fine_Tune'

    self.model.summary( show_trainable=True )


  def create_model(self) -> None:

      if self.architecture == Architecture.RESNET50V2:
        from keras.applications.resnet_v2 import ResNet50V2
        model_choose = ResNet50V2(include_top=False, weights='imagenet', input_shape=self.input_shape)

      elif self.architecture == Architecture.MOBILENETV2:
        from keras.applications.mobilenet_v2 import MobileNetV2
        model_choose = MobileNetV2(include_top=False, weights='imagenet', input_shape=self.input_shape)

      else:
        from keras.applications.vgg16 import VGG16
        model_choose = VGG16(include_top=False, weights='imagenet', input_shape=self.input_shape)


      if self.fine_tune > 0:
        for layer in model_choose.layers[:-self.fine_tune]:
                layer.trainable = False
      else:
        for layer in model_choose.layers:
                layer.trainable = True

      top_model = model_choose.output
      top_model = GlobalAveragePooling2D()(top_model)
      top_model = Dense (4096, activation='relu')(top_model)
      top_model = Dense(1072, activation='relu')(top_model)
      output_layer = Dense(self.n_classes, activation='softmax')(top_model)

      self.model = Model(inputs=model_choose.input, outputs=output_layer)

      self.model.compile(optimizer=self.optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

      return self.model



  def fit_model(self) -> None:

    start_time = time.time()

    self.history = self.model.fit( self.data_train,
                          batch_size = self.batch_size,
                          epochs = self.n_epochs,
                          validation_data = self.data_val,
                          steps_per_epoch = self.n_steps,
                          validation_steps = self.n_val_steps,
                          callbacks = self.callbacks,
                          verbose = 1)

    end_time = time.time()
    self.total_time = int(end_time - start_time)


In [ ]:
class Evaluator():

  y_pred = None

  def __init__(self, data:Datas, builder: Builder) -> None:
    self.batch_size = data.batch_size
    self.data_test = data.list_data[2]
    self.architecture = builder.architecture
    self.fine_tune = builder.fine_tune
    self.callbacks = builder.callbacks
    self.n_epochs = builder.n_epochs
    self.lr = builder.optimizer.lr
    self.total_time = builder.total_time
    self.model = builder.model
    self.history = builder.history



  def about(self) -> None:

    seconds  = self.total_time
    hour = seconds // 3600
    seconds = seconds % 3600
    min = seconds // 60
    seconds = seconds % 60


    lr = float(backend.get_value(self.lr))

    print('Last Trainning and evaluate:', datetime.datetime.now().date())
    print('Architecture:', self.architecture.name)
    print('Batch Size:', self.batch_size)
    print('Learnning Rate:', str(lr))
    print('Fine Tune: ' + ('unrealized (all layers)' if self.fine_tune == 0 else str(self.fine_tune)))
    print('Epochs covered / Epochs total: {:d} / {:d}'.format(len(self.history.history['loss']), self.n_epochs))
    print('Time Trainning: {:d}:{:d}:{:d}'.format(hour, min, seconds))

  def report(self) -> None:

    self.model.load_weights(self.callbacks[2].filepath)

    y_pred = self.model.predict(self.data_test, batch_size=self.batch_size, verbose=1)
    self.y_pred = np.argmax(y_pred, axis=1)

    report = classification_report(self.data_test.classes, self.y_pred)

    print(report)


  def metrics(self) -> None:

    print('Recall: %.3f' % recall_score(self.data_test.classes, self.y_pred, average='weighted'))
    print('Precision: %.3f' % precision_score(self.data_test.classes, self.y_pred, average='weighted'))
    print('Accuracy: %.3f' % accuracy_score(self.data_test.classes, self.y_pred))
    print('Loss %.3f' % hamming_loss(self.data_test.classes, self.y_pred))


  def confusion_matrix(self) -> None:


    y_true = self.data_test.classes
    class_names = self.data_test.class_indices.keys()


    cm_support = confusion_matrix( y_true, self.y_pred)

    #Calculo da precisao
    cm_precision = cm_support.astype('float') / cm_support.sum(axis=0)[:, np.newaxis]

    precision = ((np.asarray(cm_precision)).reshape(5,5))
    support = ((np.asarray(cm_support)).reshape(5,5))

    labels = (np.asarray(["{:d} \n  {:.2f}".format(sup,pre)
                      for pre, sup in zip(precision.flatten(),
                                               support.flatten())])).reshape(5,5)

    fig, (ax) = plt.subplots(1, figsize=(10, 10))

    sns.heatmap(
        cm_precision,
        annot=labels,
        square=True,
        xticklabels=class_names,
        yticklabels=class_names,
        fmt='',
        cmap=plt.cm.Blues,
        cbar=False,
        ax=ax
    )

    fine_tune_bool = '' if self.fine_tune == 0 else ' c/ Ajuste Fino'
    title = "Matriz de Confusão " + self.architecture.name + fine_tune_bool

    ax.set_title(title, fontsize=16)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")
    ax.set_ylabel('Classe Verdadeira', fontsize=12)
    ax.set_xlabel('Classe Prevista', fontsize=12)
    plt.show()